In [ ]:
# imports
import os
import json
import math
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import display, Markdown, update_display
import gradio as gr
load_dotenv(override=True)
os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))



In [ ]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2' 
OLLAMA_BASE_URL = "http://localhost:11434/v1"


In [ ]:
# set up environment
openai = OpenAI()
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key="ollama")


In [ ]:
# System prompt
system_prompt="""
You are a helpful assistant that takes technical questions from a student and answers them with detailed explanation.
When a student asks for a calculation or the result of a math expression (e.g. square root, powers, logarithms), use the calculate_math tool to get the exact value, then explain the result clearly.
"""


## Tool: Calculator (for math in technical Q&A)

The model can call this tool when the user asks for a calculation. This demonstrates **LLM tool/function calling**.


In [ ]:
# Calculate math tool
def calculate_math(math_equation):
    """Run a safe math expression (e.g. sqrt(25), 2**10). Used when the model calls the tool."""
    print("Tool called")
    allowed = {"__builtins__": None}
    allowed.update({k: getattr(math, k) for k in dir(math) if not k.startswith("_")})
    return eval(math_equation, allowed, {})


calculate_math_function = {
    "name": "calculate_math",
    "description": "Evaluate a math expression. Use when the user asks for a calculation or the result of an equation (e.g. sqrt(25), 2**10, log(100)). Pass only the expression as a string.",
    "parameters": {
        "type": "object",
        "properties": {
            "math_equation": {
                "type": "string",
                "description": "The math expression to evaluate, e.g. sqrt(25) or 2**10",
            },
        },
        "required": ["math_equation"],
        "additionalProperties": False,
    },
}
tools = [{"type": "function", "function": calculate_math_function}]

In [ ]:
# Handle tool call
def handle_tool_call(tool_call):
    """Execute the tool the model requested and return the message to send back."""
    arguments = json.loads(tool_call.function.arguments)
    math_equation = arguments.get("math_equation", "")
    result = calculate_math(math_equation)
    return {
        "role": "tool",
        "content": json.dumps({"math_equation": math_equation, "result": result}),
        "tool_call_id": tool_call.id,
    }

In [ ]:
# TTS
def talker(message):
    response = openai.audio.speech.create(
      model="gpt-4o-mini-tts",
      voice="onyx",
      input=message
    )
    return response.content

In [ ]:
# Chat
def chat(question, model):
    """Chat using the selected model."""
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question},
    ]

    if model == "GPT":
        response = openai.chat.completions.create(
            model=MODEL_GPT,
            messages=messages,
            tools=tools,
        )
        msg = response.choices[0].message
        response_text = msg.content or ""

        # If the model decided to call a tool, run it and get the final answer
        if response.choices[0].finish_reason == "tool_calls" and msg.tool_calls:
            messages.append({"role": "assistant", "content": msg.content or None, "tool_calls": msg.tool_calls})
            for tc in msg.tool_calls:
                messages.append(handle_tool_call(tc))
            follow = openai.chat.completions.create(
                model=MODEL_GPT,
                messages=messages,
            )
            response_text = follow.choices[0].message.content or ""
    else:
        stream = ollama.chat.completions.create(
            model=MODEL_LLAMA,
            messages=messages,
            stream=True,
        )
        response_text = ""
        for chunk in stream:
            response_text += chunk.choices[0].delta.content or ""

    audio_content = talker(response_text) if response_text else None
    return response_text or "(No text response)", audio_content


In [ ]:
# UI
with gr.Blocks() as demo:
    
    gr.Markdown("**Ask technical questions**")
    
    with gr.Row():
        question = gr.Textbox(
            placeholder="Your question...",
            lines=10,
            label="Question"
        )
    with gr.Row():
        model = gr.Dropdown(
            ["GPT", "Ollama"],
            label="Model",
            value="GPT",
            
        )
    
    btn = gr.Button("Send", variant="primary")
    
    audio = gr.Audio(autoplay=True, label="Voice (if available)")
    gr.Markdown("**Answer:**")
    output = gr.Markdown()

    # Submit on button or Enter
    question.submit(chat, [question, model], [output, audio])
    btn.click(chat, [question, model], [output, audio])

demo.launch(inline=True, height=500)